# Datan analysointi K2020_2 - Tehtävät kerta 3

Palauta Optiman palautuslaatikkoon viimeistään su 16.2. Kirjoita palautuslaatikon kommenttikenttään, mitä tehtävät olet tehnyt kokonaan/osittain.

**Tee tehtävät ilman for-silmukoita tms.**

**Opiskelijan nimi:**


## 1. Maakunnat





Lue DataFrameen tiedosto jossa on tilastotietoja Suomen kunnnista (vuosilta 2017-19). Kenttäerotin puolipiste ja deimaalierotin pilkku.

https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/kunnat.txt


a) Laske kuinka paljon maapinta-alaa on kussakin seutukunnassa ja tulosta maapinta-alaltaan 5 suurinta seutukuntaa suuruusjärjestyksessä.

b) Laske kuinka monta prosenttia kunkin maakunnan väestöstä asuu kaupungeissa (kuntamuoto) ja tulosta 5 suurinta %-osuutta. 
 
c) Laske ulkomaan ruotsinkielisten osuus maakunnittain (huom. et voi laskea prosenttiluvuista keskiarvoa, koska kuntien väkiluvut vaihtelevat). Tulosta 5 suurinta osuutta.




In [ ]:
import pandas as pd

df = pd.read_csv('https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/kunnat.txt', sep = ';', decimal=',')

#print(df)
print("a) Seutukuntien pinta-alat:")
df1 = pd.DataFrame(df['maapinta-ala'].groupby(df['seutukunta']).sum())
print(df1.sort_values('maapinta-ala', ascending=False).head(5))

#df2 = pd.DataFrame(df.groupby(df['seutukunta'])['maapinta-ala'].sum())
#df2['sum'] = df['maapinta-ala'].groupby(df['seutukunta']).sum()
#print(df2.sort_values('sum', ascending=False).head(5))

print("b) maakuntien kaupunkistuminen:")
print(df)
#df2 = pd.DataFrame(df['Väkiluku'].groupby(df['maakunta']).sum())
df2 = pd.DataFrame(df['Väkiluku'].groupby([df['maakunta'], df['kuntamuoto']]).sum())
print(df2.head(5))
df2Kaupunkilaiset = df2[(df2['kuntamuoto'] == 'Kaupunki')]
#print(df2[(df2['kuntamuoto'] == 'Kaupunki')])
print(df2Kaupunkilaiset)

## 2.  access.log

Lue erään palvelimen (3 tunnin ajan) access.log-tiedosto (15 Mt zip, noin 600 000 riviä) DataFrameen  
http://gpspekka.kapsi.fi/accesslog2.zip  
2-kerran teoriamateriaalissa on ohjeita datan lukemiseen.

Kentät  
* ip-osoite
* ident-id (-)
* userid (-)
* time
* request
* status
* size
* referer
* user agent


Parsi aikakentästä tunti ja minuutit ja laske jokaiselle minuutille kuinka monta eri ip-osoitetta on kyseisenä päivänä sivulla vieraillut.   

Tulosta listaa suuruusjärjestyksessä eli eniten eri ip-osoitteita sisältänyt minuutti ensin. 

Alkua:
```
12:55    792
12:51    789
12:54    789
12:53    787
12:56    786
12:52    783
14:00    773
14:01    769
13:59    762
12:50    760

```


In [21]:
import pandas as pd

#df = pd.read_csv('http://gpspekka.kapsi.fi/accesslog2.zip', sep = ' ')
df = pd.read_csv("http://gpspekka.kapsi.fi/accesslog2.zip", compression="zip",
                 sep = ' ', 
                 usecols = [0,3],
                 names = ['ip', 'time'],
                 converters = {'time' : (lambda x : x[1:])}
                )

df['h'] = df['time'].str.split(':').str.get(1).str.strip()#.astype(int)
df['min'] = df['time'].str.split(':').str.get(2).str.strip()#.astype(int)
#df['hour'] = df[(df['h'])]

df3 = pd.DataFrame(df['ip'].groupby([df['h'], df['min']]).nunique())
print(df3.sort_values('ip', ascending=False).head(15))

         ip
h  min     
12 55   792
   51   789
   54   789
   53   787
   56   786
   52   783
14 00   773
   01   769
13 59   762
12 50   760
13 02   756
12 58   756
   57   755
13 00   753
   57   752


## 3. Kone

Tiedostossa http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/kone.csv  (11 Mt) on erään koneen toiminnasta dataa 10 sekunnin välein. Aikaleiman lisäksi datassa on sarake `katko`, joka on 1 kun koneen toiminta on ollut katkoksissa ja 0 kun kone on ollut toiminnassa.

Poimi datasta tietoa katkoista eli **peräkkäisistä 1-riveistä**. Listaa kustakin katkosta:  
  * monesko katko on kyseessä
  * koska katko alkoi (eli ensimmäinen aikaleima)
  * koska katko loppui (eli viimeinen aikaleima)
  * kauanko katko kesti tunteina (rivien määrä jaettuna 360:lla).
  
10 ensimmäistä katkoa pitäisi näyttää tältä:
![katkot](https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/kone.png)  

In [20]:
import pandas as pd

df = pd.read_csv('http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/kone.csv', sep = ',')

df['katko_alkaa'] = (df['katko']>0) & (df['katko'].shift(1)==0)  

print(df.head(50))

print('\n-----------\n')

df['katko_nro']  = df['katko_alkaa'].cumsum()

print(df.head(10))

df['kesto'] = df[df['katko']>0].groupby('katko_nro')['katko'].count()
print(df[df['katko']>0].groupby('katko_nro')['katko'].count())

                           aika  katko  katko_alkaa
0   2019-03-18T07:00:00.000000Z      0        False
1   2019-03-18T07:00:10.000000Z      0        False
2   2019-03-18T07:00:20.000000Z      0        False
3   2019-03-18T07:00:30.000000Z      0        False
4   2019-03-18T07:00:40.000000Z      0        False
5   2019-03-18T07:00:50.000000Z      0        False
6   2019-03-18T07:01:00.000000Z      0        False
7   2019-03-18T07:01:10.000000Z      0        False
8   2019-03-18T07:01:20.000000Z      0        False
9   2019-03-18T07:01:30.000000Z      0        False
10  2019-03-18T07:01:40.000000Z      0        False
11  2019-03-18T07:01:50.000000Z      0        False
12  2019-03-18T07:02:00.000000Z      0        False
13  2019-03-18T07:02:10.000000Z      0        False
14  2019-03-18T07:02:20.000000Z      0        False
15  2019-03-18T07:02:30.000000Z      0        False
16  2019-03-18T07:02:40.000000Z      0        False
17  2019-03-18T07:02:50.000000Z      0        False
18  2019-03-

## 4. Lumeton alkutalvi?

Lataa DataFrameen csv-tiedosto  
http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/saajkl20200118.csv

Tiedostossa on päivittäisiä säähavaintoja Jyväskylän lentoasemalta vuodesta 1959.

Laske tietojen perusteella jokaiselle "alkutalvelle" 1. marraskuuta - 18. tammikuuta  **"lumisumma"** jossa on alkutalven jokaisen vuorokauden lumensyvyys-lukema laskettu yhteen (ei ole mikään oikea meteorologinen termi).

Huomioita:
* lumisumma pitää laske talvittain, ei siis vuosittain. Tee uusi sarake joka kertoo mihin (alku)talveen kyseinen vuorokausi kuuluu. Siis tammikuu 2020 kuuluu samaan alkutalveen kuin joulukuu 2019.

* Ilmatieteen laitos käyttää arvoa -1 lumensyvyyden arvona jos "lunta ei ole näkyvissäkään". Ei haluta kuitenkaan miinustaa yhtä näinä päivinä, joten korvaa -1:t nollilla.

* Lisäksi joinain päivinä lumensyvyys todella (jostain syystä) puuttuu. Jos nämä jätetään laskusta pois, kyseisen talven lukema jää liian pieneksi (jos lunta on kuitenkin ollut). Lienee turvallista olettaa että lunta on varmaan ollut suunnilleen saman verran kuin edellisenä päivänä, joten täytä NaN-arvot edellsillä arvoilla

Anna tuloksena DataFrame, jossa on indeksinä talvi (joko muodossa 2019 tai 2019-2020) ja sarakkeina 
* alkutalventalven lumisumma 
* sijoitus lumisummien joukossa (lumisin alkutalvi = 1) 
* alkutalven lumipäivien lukumäärä (eli kuinka monena päivänä lumensyvyys on ollut suurempi kuin nolla)
* alkutalven suurin lumensyvyys.

mallia alusta ja lopusta:  
 ```
           Lumensyvyys  sija  lumipaivia  maksimi
Talvi                                            
1959-1960         1292    20          61       38
1960-1961         1305    19          61       52
1961-1962         1116    28          54       41
1962-1963         1065    30          65       35
1963-1964          850    41          69       22
           Lumensyvyys  sija  lumipaivia  maksimi
Talvi                                            
2015-2016          139    61          31       14
2016-2017          737    50          69       18
2017-2018         1476    15          62       49
2018-2019          568    53          39       31
2019-2020          856    39          68       30
 ```


In [18]:
import pandas as pd

df = pd.read_csv('http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/saajkl20200118.csv', sep = ',')

df.loc[(df['Lumensyvyys'] < 0), 'Lumensyvyys'] = 0
df.loc[(df['Lumensyvyys'].isnull()), 'Lumensyvyys'] = df.shift(1)['Lumensyvyys']


print(df.sort_values(['Kk'], ascending=False).head(15))

      Vuosi  Kk  Pv    Klo Aikavyöhyke  Sademäärä  Lumensyvyys  \
6029   1975  12   3  00:00         UTC        3.1          3.0   
3130   1967  12  26  00:00         UTC       -1.0         15.0   
3132   1967  12  28  00:00         UTC        4.8         15.0   
3133   1967  12  29  00:00         UTC        3.2         20.0   
3134   1967  12  30  00:00         UTC        0.4         25.0   
...     ...  ..  ..    ...         ...        ...          ...   
9332   1984  12  18  00:00         UTC        0.7         11.0   
9331   1984  12  17  00:00         UTC        0.3         10.0   
9330   1984  12  16  00:00         UTC        0.1         10.0   
9329   1984  12  15  00:00         UTC       -1.0         10.0   
9328   1984  12  14  00:00         UTC        0.1         10.0   

      Ilman lämpötila  Ylin lämpötila  Alin lämpötila  
6029              1.9             2.5            -0.4  
3130            -20.8           -16.5           -26.4  
3132            -15.2            -7.3  

## 5. Liiga

Tiedostossa http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/liiga.txt on SM-liigan (nyk. Liiga) runkosarjan ottelutukset 2010-luvulta.  

huom. -sarakkeessa on merkintä JA tai VL, jos ottelu on ratkennut vasta jatkoajalla tai voittomaalikilpailussa.

Voittanut (enemmän maaleja tehnyt joukkue) saa 3 pistettä ja hävinnyt 0 pistettä, paitsi jos ottelu on ratkennut vasta jatkoajalla tai voittomaalikilpailussa, voittanut joukkue saa 2 pistettä ja hävinnyt yhden pisteen.



Generoi tulosten perusteella "vuosikymmenen sarjataulukko", jossa on kullekin joukkueelle omana sarakkeenaan
* sijoitus
* joukkueen nimi
* pelatut ottelut
* 3 pisteen voitot
* 2 pisteen voitot (JA/VL)
* 1 pisteen tappiot (JA/VL)
* 0 pisteen tappiot
* tehdyt maalit ja päästetyt maalit (samassa sarakkeessa)
* pisteet



Lajittele sarjataulukko pisteiden mukaan (eniten pisteitä ekana). Tasapisteissä olevat joukkueet menevät maalieron (tehdyt maalit - päästetyt maalit) mukaiseen järjestykseen, ja jos sekin on sama niin tehdyt maalit ratkaisevat.

tavoiteltu lopputulos:

```
    sija   joukkue  ott   v3  v2  t1   t0     maalit  pist
0      1    Kärpät  574  295  70  51  158  1678-1244  1076
1      2       JYP  575  277  62  58  178  1614-1311  1013
2      3   Tappara  575  261  78  62  174  1598-1387  1001
3      4      HIFK  575  264  62  63  186  1637-1403   979
4      5     KalPa  574  224  69  68  213  1414-1400   878
5      6     Lukko  574  226  62  67  219  1516-1479   869
6      7     Ässät  574  215  64  56  239  1428-1501   829
7      8  Pelicans  574  215  57  56  246  1530-1595   815
8      9       HPK  575  198  80  60  237  1416-1498   814
9     10     SaiPa  574  201  66  71  236  1451-1576   806
10    11       TPS  572  193  59  78  242  1410-1596   775
11    12     Ilves  574  192  55  60  267  1426-1643   746
12    13     Blues  360  126  43  36  155    852-943   500
13    14   Jokerit  240  102  29  37   72    672-586   401
14    15     Sport  334   87  37  47  163   810-1028   382
15    16    KooKoo  275   79  32  42  122    664-809   343
16    17   Jukurit  215   56  21  34  104    486-603   244
```

(jos saat Ässille 832 pistettä, ohjelmakoodisi tulkitsee voittajan tuloksesta 11-3 väärin...) 


In [1]:
import pandas as pd

df = pd.read_csv('http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/liiga.txt', sep = ',')

df['KotiJ'] = df['ottelu'].str.split('-').str.get(0).str.strip()
df['VierasJ'] = df['ottelu'].str.split('-').str.get(1).str.strip()

df['KotiM'] = df['tulos'].str.split('-').str.get(0).str.strip().astype(int)
df['VierasM'] = df['tulos'].str.split('-').str.get(1).str.strip().astype(int)

df['KotiP'] = 0  # "alustetaan" pisteet nolliksi
df['VierasP'] = 0
df['v3'] = 0
df['v2'] = 0
df['v1'] = 0
df['v0'] = 0
df['k3'] = 0
df['k2'] = 0
df['k1'] = 0
df['k0'] = 0

df.loc[(df['huom'].isnull()) & (df['KotiM']>df['VierasM']), 'KotiP'] = 3 # kotivoitosta 3 pistettä jos NaN
df.loc[(df['huom'].notnull()) & (df['KotiM']>df['VierasM']), 'KotiP'] = 2 # kotivoitosta 2 pistettä jos JA/VL
df.loc[(df['huom'].notnull()) & (df['KotiM']<df['VierasM']), 'KotiP'] = 1 # kotitappiosta 1 piste jos JA/VL

df.loc[(df['huom'].isnull()) & (df['KotiM']<df['VierasM']), 'VierasP'] = 3 # vierasvoitosta 3 pistettä jos NaN
df.loc[(df['huom'].notnull()) & (df['KotiM']<df['VierasM']), 'VierasP'] = 2 # vierasvoitosta 2 pistettä jos JA/VL
df.loc[(df['huom'].notnull()) & (df['KotiM']>df['VierasM']), 'VierasP'] = 1 # vierastappiosta 1 piste jos JA/VL

#------------

df.loc[(df['huom'].isnull()) & (df['KotiM']>df['VierasM']), 'k3'] = 1 # kotivoitosta 3 pistettä jos NaN
df.loc[(df['huom'].notnull()) & (df['KotiM']>df['VierasM']), 'k2'] = 1 # kotivoitosta 2 pistettä jos JA/VL
df.loc[(df['huom'].notnull()) & (df['KotiM']<df['VierasM']), 'k1'] = 1 # kotitappiosta 1 piste jos JA/VL
df.loc[(df['huom'].isnull()) & (df['KotiM']<df['VierasM']), 'k0'] = 1 # kotitappio

df.loc[(df['huom'].isnull()) & (df['KotiM']<df['VierasM']), 'v3'] = 1 # vierasvoitosta 3 pistettä jos NaN
df.loc[(df['huom'].notnull()) & (df['KotiM']<df['VierasM']), 'v2'] = 1 # vierasvoitosta 2 pistettä jos JA/VL
df.loc[(df['huom'].notnull()) & (df['KotiM']>df['VierasM']), 'v1'] = 1 # vierastappiosta 1 piste jos JA/VL
df.loc[(df['huom'].isnull()) & (df['KotiM']>df['VierasM']), 'v0'] = 1 # vierastappio

#print(df.head(10))

df2 = pd.DataFrame((df['KotiJ'].groupby([df['KotiJ']]).unique()))
#df2.columns = ['Joukkue']
df2['Joukkue'] = df2.iloc[:, 0].apply(''.join)
df2['Ottelut'] = (df['KotiJ'].groupby([df['KotiJ']]).count()) + (df['VierasJ'].groupby([df['VierasJ']]).count())
df2['V3'] = ((df['k3'].groupby([df['KotiJ']]).sum()) + (df['v3'].groupby([df['VierasJ']]).sum()))
df2['V2'] = ((df['k2'].groupby([df['KotiJ']]).sum()) + (df['v2'].groupby([df['VierasJ']]).sum()))
df2['T1'] = ((df['k1'].groupby([df['KotiJ']]).sum()) + (df['v1'].groupby([df['VierasJ']]).sum()))
df2['T0'] = ((df['k0'].groupby([df['KotiJ']]).sum()) + (df['v0'].groupby([df['VierasJ']]).sum()))
df2['Pisteet'] = ((df['KotiP'].groupby([df['KotiJ']]).sum()) + (df['VierasP'].groupby([df['VierasJ']]).sum()))
df2['TM'] = ((df['KotiM'].groupby([df['KotiJ']]).sum()) + (df['VierasM'].groupby([df['VierasJ']]).sum()))
df2['PM'] = ((df['VierasM'].groupby([df['KotiJ']]).sum()) + (df['KotiM'].groupby([df['VierasJ']]).sum()))
df2['Maaliero'] = df2['TM'] - df2['PM']
df2['Maalit'] = df2['TM'].astype(str) + "-" + df2['PM'].astype(str)
#print(df2)

tulostettavat = ['Sija', 'Joukkue', 'Ottelut', 'V3', 'V2', 'T1', 'T0', 'Maalit', 'Pisteet']
df2 = df2.sort_values(['Pisteet', 'Maaliero', 'TM'], ascending=False)
df2.insert(0, 'Sija', range(1, 1 + len(df2)))
df2.set_index('Sija')
print(df2[tulostettavat])




          Sija   Joukkue  Ottelut   V3  V2  T1   T0     Maalit  Pisteet
KotiJ                                                                  
Kärpät       1    Kärpät      574  295  70  51  158  1678-1244     1076
JYP          2       JYP      575  277  62  58  178  1614-1311     1013
Tappara      3   Tappara      575  261  78  62  174  1598-1387     1001
HIFK         4      HIFK      575  264  62  63  186  1637-1403      979
KalPa        5     KalPa      574  224  69  68  213  1414-1400      878
Lukko        6     Lukko      574  226  62  67  219  1516-1479      869
Ässät        7     Ässät      574  215  64  56  239  1428-1501      829
Pelicans     8  Pelicans      574  215  57  56  246  1530-1595      815
HPK          9       HPK      575  198  80  60  237  1416-1498      814
SaiPa       10     SaiPa      574  201  66  71  236  1451-1576      806
TPS         11       TPS      572  193  59  78  242  1410-1596      775
Ilves       12     Ilves      574  192  55  60  267  1426-1643  